## Observations and Insights 

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

%matplotlib notebook

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset

# Display the data table for preview


In [2]:
mouse_metadata.head()


,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g)
0,k403,Ramicane,Male,21,16
1,s185,Capomulin,Female,3,17
2,x401,Capomulin,Female,16,15
3,m601,Capomulin,Male,22,17
4,g791,Ramicane,Male,11,16


In [3]:
study_results.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,b128,0,45.0,0
1,f932,0,45.0,0
2,g107,0,45.0,0
3,a457,0,45.0,0
4,c819,0,45.0,0


In [7]:
mouse_metadata.shape


(249, 5)

In [8]:
study_results.shape

(1893, 4)

In [10]:
# Checking the number of mice.
#make sure there are no duplicates in the mouse_metadata file
total_mouse = len((mouse_metadata["Mouse ID"]).unique())
total_mouse

249

In [15]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
Duplicated_study = study_results[study_results.duplicated(['Mouse ID', 'Timepoint'], keep = False)]
Duplicated_study

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites
107,g989,0,45.000000,0
137,g989,0,45.000000,0
329,g989,5,48.786801,0
360,g989,5,47.570392,0
620,g989,10,51.745156,0
681,g989,10,49.880528,0
815,g989,15,51.325852,1
869,g989,15,53.442020,0
950,g989,20,55.326122,1
1111,g989,20,54.657650,1


In [16]:
# Optional: Get all the data for the duplicate mouse ID. 
Duplicated_study.shape


(10, 4)

In [14]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
clean_study = study_results.drop_duplicates(subset = ['Mouse ID','Timepoint'], keep = False)
clean_study.shape
#the keep = false means that we are dropping all rows associated with these duplicates

(1883, 4)

In [19]:
# Checking the number of mice in the clean DataFrame
total_mouse_clean = len((clean_study["Mouse ID"]).unique())
total_mouse_clean

249

In [20]:
# join the mouse_metadata and the clean_study into one dataframe for analysis purposes
clean_study_merge = pd.merge(clean_study, mouse_metadata, on="Mouse ID", how = "left")
clean_study_merge.shape

(1883, 8)

In [21]:
clean_study_merge.head(20)

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.0,0,Capomulin,Female,9,22
1,f932,0,45.0,0,Ketapril,Male,15,29
2,g107,0,45.0,0,Ketapril,Female,2,29
3,a457,0,45.0,0,Ketapril,Female,11,30
4,c819,0,45.0,0,Ketapril,Male,21,25
5,h246,0,45.0,0,Ketapril,Male,13,30
6,p189,0,45.0,0,Ketapril,Male,8,28
7,n923,0,45.0,0,Ketapril,Male,19,30
8,q119,0,45.0,0,Ketapril,Male,17,30
9,f993,0,45.0,0,Naftisol,Male,21,28


In [22]:
clean_study_merge.columns


Index(['Mouse ID', 'Timepoint', 'Tumor Volume (mm3)', 'Metastatic Sites',
       'Drug Regimen', 'Sex', 'Age_months', 'Weight (g)'],
      dtype='object')

In [25]:
clean_study_merge.dtypes


Mouse ID               object
Timepoint               int64
Tumor Volume (mm3)    float64
Metastatic Sites        int64
Drug Regimen           object
Sex                    object
Age_months              int64
Weight (g)              int64
dtype: object

In [34]:
clean_study_merge.rename(columns = {"Tumor Volume (mm3)":"TV"}, inplace = True)
clean_study_merge.columns

Index(['Mouse ID', 'Timepoint', 'TV', 'Metastatic Sites', 'Drug Regimen',
       'Sex', 'Age_months', 'Weight (g)'],
      dtype='object')

## Summary Statistics

In [50]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.

clean_study_grouped = clean_study_merge.groupby(['Drug Regimen'])
#study_count = clean_study_grouped['Drug Regimen'].count()
#study_count
study_mean = clean_study_grouped['TV'].mean()
#study_mean
study_median = clean_study_grouped["TV"].median()
#study_median
study_var = clean_study_grouped["TV"].var()
#study_var
study_std = clean_study_grouped["TV"].std()

study_sem = clean_study_grouped["TV"].sem()


drug_regimen_properties = pd.DataFrame({"Mean":study_mean,
                                        "Median":study_median,
                                        "Variance":study_var,
                                        "Standard Deviation":study_std,
                                        "SEM":study_sem                                        
                                        })
drug_regimen_properties

,Mean,Median,Variance,Standard Deviation,SEM
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,52.458254,50.854632,44.053659,6.637293,0.540135
Ramicane,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,54.233149,52.431737,59.450562,7.710419,0.573111


In [52]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line


drug_regimen_properties

AttributeError: 'float' object has no attribute 'var'

## Bar and Pie Charts

In [9]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pandas.



In [10]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pyplot.



In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [14]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [15]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [16]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [17]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [18]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
